In [100]:
import pandas as pd
import numpy as np

In [101]:
pwd

'/Users/ioanwilliams/notebooks/Recommendation Systems/ml-latest-small'

In [102]:
movies = pd.read_csv(r"movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [103]:
all_ratings = pd.read_csv(r"all_ratings.csv")
all_ratings.tail()

,Unnamed: 0,userId,movieId,rating
100732,728,672,148626,4.0
100733,729,672,150548,3.0
100734,730,672,152091,1.0
100735,731,672,159093,3.0
100736,732,672,160438,3.0


In [104]:
all_ratings.count()

Unnamed: 0    100737
userId        100737
movieId       100737
rating        100737
dtype: int64

In [105]:
R_df = all_ratings.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
R_df.tail()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
670,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
671,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
672,3.0,2.5,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
ratings_matrix = R_df.values
#user_ratings_mean = np.mean(ratings_matrix, axis = 1)
#R_demeaned = ratings_matrix - user_ratings_mean.reshape(-1, 1)

In [107]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(ratings_matrix, k = 50)
sigma = np.diag(sigma)

In [108]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) #+ user_ratings_mean.reshape(-1, 1)

In [109]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False) # UserID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.userId == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [110]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [111]:
already_rated, predictions = recommend_movies(preds_df, 672, movies, all_ratings, 15)

User 672 has already rated 733 movies.
Recommending highest 15 predicted ratings movies not already rated.


In [112]:
predictions

,movieId,title,genres
1121,1625,"Game, The (1997)",Drama|Mystery|Thriller
2867,4022,Cast Away (2000),Drama
2410,3408,Erin Brockovich (2000),Drama
2774,3897,Almost Famous (2000),Drama
6628,68157,Inglourious Basterds (2009),Action|Drama|War
2999,4226,Memento (2000),Mystery|Thriller
3587,5218,Ice Age (2002),Adventure|Animation|Children|Comedy
4538,7143,"Last Samurai, The (2003)",Action|Adventure|Drama|War
4942,8529,"Terminal, The (2004)",Comedy|Drama|Romance
904,1288,This Is Spinal Tap (1984),Comedy


In [113]:
already_rated.head()

,Unnamed: 0,userId,movieId,rating,title,genres
578,578,672,64957,5.0,"Curious Case of Benjamin Button, The (2008)",Drama|Fantasy|Mystery|Romance
12,12,672,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
438,438,672,5825,5.0,"Life and Death of Colonel Blimp, The (1943)",Drama|Romance|War
257,257,672,2762,5.0,"Sixth Sense, The (1999)",Drama|Horror|Mystery
38,38,672,296,5.0,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
